In [1]:
%matplotlib notebook
import pandas as pd
import numpy as np
import math
import statistics
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder, normalize
import matplotlib.pyplot as plt
import seaborn as sn
import matplotlib
from mlxtend.plotting import plot_confusion_matrix
import warnings
from scipy.signal import savgol_filter, find_peaks, correlate
from sklearn.decomposition import PCA
import scipy.fftpack as FFT
from sklearn.utils import shuffle
import joblib
warnings.filterwarnings('ignore')
matplotlib.rcParams['figure.figsize'] = [12,5]

## help functions

In [2]:
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

def convert_to_datetime(x):
    dt = datetime.fromtimestamp(x // 1000000000)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')
    s += '.' + str(int(x % 1000000000)).zfill(9)
    return s

def heatmap(cm, labels):
    df_cm = pd.DataFrame(cm, index=labels, columns=labels)
    plt.figure(figsize = (10,7))
    sn.heatmap(df_cm, annot=True)

    plt.show()
    
def preprocess(data, activity, drop_interval_begin = 3, drop_interval_end = 3):
    #convert nanoseconds to date 
    data['time'] = data['time'].apply(convert_to_datetime) 
    data['time'] = pd.to_datetime(data['time'])

#convert to same data type
    for i in range(1, 4): 
        data[column_names_org[i]] = data[column_names_org[i]].apply(convert_to_float)
    
#drop rows with NaN values 
    data.dropna(axis=0, how='any', inplace=True) #TODO: invullen met mean, mod of median / interpolatie

#drop duplicates 
    data.drop_duplicates(subset=None, keep='first', inplace = True)
    
    #drop first and last 3 sec
    indexFirst = data[ (data['time'].iloc[0]+ pd.to_timedelta(drop_interval_begin, unit='s')) > data['time'] ].index
    data.drop(indexFirst , inplace=True)
    indexLast = data[ (data['time'].iloc[-1]- pd.to_timedelta(drop_interval_end, unit='s')) < data['time'] ].index
    data.drop(indexLast , inplace=True)
    
    #add activity label
    data['activity'] = activity
    
    return data

#feature extraction
def get_mean_window(df):
    return df.mean()

def get_min_window(df):
    return df.min()

def get_max_window(df):
    return df.max()

def get_std_window(df):
    return df.std()

def get_med_window(df):
    return df.median()

#TODO: scipy integrate
#TODO: voor elke as apart??
#measure of activity level (m/s²)
def get_signal_magnitude_area(df):
        sum = 0
        for i in range(0, len(df)):
            sum += (abs(df['x'].iloc[i]) + abs(df['y'].iloc[i]) + abs(df['z'].iloc[i]))
        return sum /len(df)
    #result = integrate.quad(lambda t: df['x'].apply(lambda n : abs(n)) + df['y'].apply(lambda n : abs(n)) + df['z'].apply(lambda n : abs(n)), 0, len(df))

#TODO: voor elke as apart??
#degree of movement intensity (m/s²)
def get_signal_magnitude_vector(df):
    sum = 0
    for i in range(0, len(df)):
        sum += math.sqrt(df['x'].iloc[i] * df['x'].iloc[i] + df['y'].iloc[i] * df['y'].iloc[i] + df['z'].iloc[i] * df['z'].iloc[i])
    return sum

#average angle (radian) between accelerometer vector and x as (parallel with arm)
def get_tilt_angle(df):
    df_cos = pd.DataFrame(columns=["tilt_ang"])
    df_dot = df['x']
    for i in range(0, len(df)):
        s = pd.Series({"tilt_ang" : (df_dot.iloc[i])/(math.sqrt(df['x'].iloc[i]*df['x'].iloc[i] + df['y'].iloc[i]*df['y'].iloc[i] + df['z'].iloc[i]*df['z'].iloc[i]))})
        df_cos=df_cos.append(s, ignore_index=True)
    df_angle = np.arccos(df_cos)
    return df_angle.mean()['tilt_ang']

def get_power_spectral_density(df):
    df_psd = np.abs(df)**2
    return df_psd.sum()

#TODO: datatype is object en niet compex nr
def get_entropy(df):
    entropy = []
    pdf = df / df.sum()
    for i in range (1, len(pdf.columns)):
        entropy.append(np.complex(-np.nansum(pdf.iloc[:,i] * np.log2(pdf.iloc[:,i]))))
    return entropy

#generate windows with 50% overlap
def windows(df, time, overlap):
    start = df.iloc[0]
    while  (start + pd.to_timedelta(time, unit='s')) < df.iloc[-1]:
        yield start, (start + pd.to_timedelta(time, unit='s'))
        if overlap:
            start += pd.to_timedelta(time/2, unit='s')
        else:
            start += pd.to_timedelta(time, unit='s')
    #last samples 
    yield (df.iloc[-1] - pd.to_timedelta(time, unit='s')), df.iloc[-1]
        
def feature_extraction_segmentation(data, window, overlap):
    column_names = ["x_mean", "y_mean", "z_mean", "x_min", "y_min", "z_min", "x_max", "y_max", "z_max",
                "x_std", "y_std", "z_std", "x_med", "y_med", "z_med", "activity"]
    df = pd.DataFrame(columns = column_names)

    for (start, end) in windows(data['time'], window, overlap):
        vw1 = data['time'] >= start
        vw2 = data['time'] < end
        mean = get_mean_window(data[vw1 & vw2])
        min = get_min_window(data[vw1 & vw2])
        max = get_max_window(data[vw1 & vw2])
        std = get_std_window(data[vw1 & vw2])
        med = get_med_window(data[vw1 & vw2])
        sma = get_signal_magnitude_area(data[vw1 & vw2])
        smv = get_signal_magnitude_vector(data[vw1 & vw2])
        tilt = get_tilt_angle(data[vw1 & vw2])
        #fourrier transform
        t_x = data[vw1 & vw2][['time','x']].set_index('time')
        t_y = data[vw1 & vw2][['time','y']].set_index('time')
        t_z = data[vw1 & vw2][['time', 'z']].set_index('time')

        df_f = pd.DataFrame(columns=['f', 'x_f', 'y_f', 'z_f'])

    #TODO: determine sampling rate (datapoints per second)
        sampling_rate = 52

        df_f['x_f'] = FFT.fft(t_x).ravel()
        df_f['y_f'] = FFT.fft(t_y).ravel()
        df_f['z_f'] = FFT.fft(t_z).ravel()
        df_f['f'] = FFT.fftfreq(len(df_f['x_f'])) * sampling_rate

        psd = get_power_spectral_density(df_f)
        #entropy = get_entropy(df_f)

        df = df.append(pd.Series({'x_mean': mean['x'], 'y_mean': mean['y'], 'z_mean': mean['z'], "x_min" : min['x'],
                              "y_min" : min['y'], "z_min" : min['z'], "x_max" : max['x'], "y_max" : max['y'], "z_max" : max['z'],
                              "x_std" : std['x'], "y_std" : std['y'], "z_std" : std['z'], "x_med" : med['x'], "y_med" : med['y'], "z_med" : med['z'],
                              "sma" : sma, "smv" : smv, "tilt" : tilt, "x_psd" : psd['x_f'], "y_psd" : psd['y_f'], "z_psd" : psd['z_f']
                              , "activity" : data['activity'].iloc[0]}), ignore_index=True) # "x_entropy" : entropy[0], "y_entropy" : entropy[1], "z_entropy" : entropy[2],
    return df

#3 keer savgol filter uitvoeren met 33,5
def get_turns_smooth_side_swing(df):
    for i in range(0,5):
        df['x'] = savgol_filter(df[ 'x'].to_numpy(), 101, 5)
        df['y'] = savgol_filter(df[ 'y'].to_numpy(), 101, 5)
        df['z'] = savgol_filter(df[ 'z'].to_numpy(), 101, 5)
        df.plot(x='time', subplots=True)

    wx = find_peaks(df['x'])
    wy = find_peaks(df['y'])
    wz = find_peaks(df['z'])
    return (len(wx[0]) + len(wy[0]) + len(wz[0]))/3


column_names_org = ['time', 'x', 'y', 'z']

# <font color='green' >PROEFPERSOON 1

## jump_slow

In [192]:
jump_slow_left_forward_1 = []
jump_slow_right_forward_1 = []
jump_slow_left_backwards_1 = []
jump_slow_right_backwards_1 = []

#### jump_slow_left_forward

In [193]:
jump_slow3 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon1\jump_slow_left_forward_tim.csv", sep=';', header=0, skipinitialspace=True)

jump_slow3 = preprocess(jump_slow3, "jump_slow")

jump_slow_left_forward_1 = [jump_slow3]

#### jump_slow_right_forward

In [194]:
jump_slow_rigth = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon1\jump_slow_tim.csv", sep=';', header=0, skipinitialspace=True)

jump_slow_rigth = preprocess(jump_slow_rigth, "jump_slow")

jump_slow_right_forward_1 = [jump_slow_rigth]

#### jump_slow_left_backwards

In [195]:
jump_slow_left_backwards1 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon1\jump_slow_left_backwards_tim1.csv", sep=';', header=0, skipinitialspace=True)
jump_slow_left_backwards2 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon1\jump_slow_left_backwards_tim2.csv", sep=';', header=0, skipinitialspace=True)
jump_slow_left_backwards3 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon1\jump_slow_left_backwards_tim3.csv", sep=';', header=0, skipinitialspace=True)

jump_slow_left_backwards1 = preprocess(jump_slow_left_backwards1, "jump_slow", 3, 5)
jump_slow_left_backwards2 = preprocess(jump_slow_left_backwards2, "jump_slow")
jump_slow_left_backwards3 = preprocess(jump_slow_left_backwards3, "jump_slow")

jump_slow_left_backwards_1 = [jump_slow_left_backwards1, jump_slow_left_backwards2, jump_slow_left_backwards3]

#### jump_slow_right_backwards

In [196]:
jump_slow_right_backwards1 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon1\jump_slow_right_backwards_tim.csv", sep=';', header=0, skipinitialspace=True)
jump_slow_right_backwards2 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon1\jump_slow_right_backwards_tim1.csv", sep=';', header=0, skipinitialspace=True)

jump_slow_right_backwards1 = preprocess(jump_slow_right_backwards1, "jump_slow")
jump_slow_right_backwards2 = preprocess(jump_slow_right_backwards2, "jump_slow")

jump_slow_right_backwards_1 = [jump_slow_right_backwards1, jump_slow_right_backwards2]

### SEGMENTATION

In [197]:
#segmentation + feature extraction
for i in range(0, len(jump_slow_left_forward_1)):
    jump_slow_left_forward_1[i] =  feature_extraction_segmentation(jump_slow_left_forward_1[i],1, True)
    
for i in range(0, len(jump_slow_right_forward_1)): 
    jump_slow_right_forward_1[i] =  feature_extraction_segmentation(jump_slow_right_forward_1[i],1, True)
    
for i in range(0, len(jump_slow_left_backwards_1)):
    jump_slow_left_backwards_1[i] =  feature_extraction_segmentation(jump_slow_left_backwards_1[i],1, True)

for i in range(0, len(jump_slow_right_backwards_1)):
    jump_slow_right_backwards_1[i] =  feature_extraction_segmentation(jump_slow_right_backwards_1[i],1, True)


In [198]:
jump_slow_left_backwards_1 = pd.concat(jump_slow_left_backwards_1, ignore_index=True)
jump_slow_left_backwards_1.shape

(80, 22)

In [199]:
jump_slow_right_backwards_1 = pd.concat(jump_slow_right_backwards_1, ignore_index=True)
jump_slow_right_backwards_1.shape

(111, 22)

In [200]:
jump_slow_left_forward_1 = pd.concat(jump_slow_left_forward_1, ignore_index=True)
jump_slow_left_forward_1.shape

(33, 22)

In [201]:
jump_slow_right_forward_1 = pd.concat(jump_slow_right_forward_1, ignore_index=True)
jump_slow_right_forward_1.shape

(70, 22)

### SHUFFLE - BALANCING

In [ ]:
shape = np.min([
    jump_slow_left_backwards_1.shape[0], 
    jump_slow_right_backwards_1.shape[0], 
    jump_slow_left_forward_1.shape[0], 
    jump_slow_right_forward_1.shape[0]])

In [ ]:
jump_slow_left_backwards_1 = shuffle(jump_slow_left_backwards_1).head(shape).copy()
jump_slow_right_backwards_1 = shuffle(jump_slow_right_backwards_1).head(shape).copy()
jump_slow_left_forward_1 = shuffle(jump_slow_left_forward_1).head(shape).copy()
jump_slow_right_forward_1 = shuffle(jump_slow_right_forward_1).head(shape).copy()

In [ ]:
jump_slow_1 = pd.concat([jump_slow_left_backwards_1, jump_slow_right_backwards_1, jump_slow_left_forward_1, jump_slow_right_forward_1])

# <font color='green' >PROEFPERSOON 2

In [250]:
jump_slow_left_forward_2 = []
jump_slow_right_forward_2 = []
jump_slow_left_backwards_2 = []
jump_slow_right_backwards_2 = []

#### jump_slow_left_forward

In [251]:
jump_slow = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon2\jump_slow.csv", sep=';', header=0, skipinitialspace=True)
jump_slow2 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon2\jump_slow1 (meting2).csv", sep=';', header=0, skipinitialspace=True)

jump_slow = preprocess(jump_slow, "jump_slow")
jump_slow2 = preprocess(jump_slow2, "jump_slow")

jump_slow_left_forward_2 = [jump_slow, jump_slow2]

#### jump_slow_right_forward

In [252]:
jump_slow_rigth = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon2\jump_slow_right1.csv", sep=';', header=0, skipinitialspace=True)
jump_slow_rigth2 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon2\jump_slow_right2.csv", sep=';', header=0, skipinitialspace=True)
jump_slow_rigth3 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon2\jumpslow_right_forward.csv", sep=';', header=0, skipinitialspace=True)
jump_slow_rigth4 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon2\jumpslow_right_forward2.csv", sep=';', header=0, skipinitialspace=True)

jump_slow_rigth = preprocess(jump_slow_rigth, "jump_slow")
jump_slow_rigth2 = preprocess(jump_slow_rigth2, "jump_slow")
jump_slow_rigth3 = preprocess(jump_slow_rigth3, "jump_slow")
jump_slow_rigth4 = preprocess(jump_slow_rigth4, "jump_slow")

jump_slow_right_forward_2 = [jump_slow_rigth, jump_slow_rigth2, jump_slow_rigth3, jump_slow_rigth4]

#### jump_slow_left_backwards

In [253]:
jump_slow_turn_diff = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon2\jump_slow_turn_diff.csv", sep=';', header=0, skipinitialspace=True)

jump_slow_turn_diff = preprocess(jump_slow_turn_diff, "jump_slow")

jump_slow_left_backwards_2 = [jump_slow_turn_diff]

#### jump_slow_right_backwards

In [254]:
jump_slow_right_backwards1 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon2\jump_slow_right_backwards1.csv", sep=';', header=0, skipinitialspace=True)
jump_slow_right_backwards2 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon2\jump_slow_right_backwards2.csv", sep=';', header=0, skipinitialspace=True)

jump_slow_right_backwards1 = preprocess(jump_slow_right_backwards1, "jump_slow")
jump_slow_right_backwards2 = preprocess(jump_slow_right_backwards2, "jump_slow")

jump_slow_right_backwards_2 = [jump_slow_right_backwards1, jump_slow_right_backwards2]

### SEGMENTATION

In [255]:
#segmentation + feature extraction
for i in range(0, len(jump_slow_left_forward_2)):
    jump_slow_left_forward_2[i] =  feature_extraction_segmentation(jump_slow_left_forward_2[i],1, True)
    
for i in range(0, len(jump_slow_right_forward_2)): 
    jump_slow_right_forward_2[i] =  feature_extraction_segmentation(jump_slow_right_forward_2[i],1, True)
    
for i in range(0, len(jump_slow_left_backwards_2)):
    jump_slow_left_backwards_2[i] =  feature_extraction_segmentation(jump_slow_left_backwards_2[i],1, True)

for i in range(0, len(jump_slow_right_backwards_2)):
    jump_slow_right_backwards_2[i] =  feature_extraction_segmentation(jump_slow_right_backwards_2[i],1, True)


In [256]:
jump_slow_left_backwards_2 = pd.concat(jump_slow_left_backwards_2, ignore_index=True)
jump_slow_left_backwards_2.shape

(109, 22)

In [257]:
jump_slow_right_backwards_2 = pd.concat(jump_slow_right_backwards_2, ignore_index=True)
jump_slow_right_backwards_2.shape

(194, 22)

In [258]:
jump_slow_left_forward_2 = pd.concat(jump_slow_left_forward_2, ignore_index=True)
jump_slow_left_forward_2.shape

(110, 22)

In [259]:
jump_slow_right_forward_2 = pd.concat(jump_slow_right_forward_2, ignore_index=True)
jump_slow_right_forward_2.shape

(266, 22)

### SHUFFLE - BALANCING

In [ ]:
shape = np.min([
    jump_slow_left_backwards_2.shape[0], 
    jump_slow_right_backwards_2.shape[0], 
    jump_slow_left_forward_2.shape[0], 
    jump_slow_right_forward_2.shape[0]])

In [ ]:
jump_slow_left_backwards_2 = shuffle(jump_slow_left_backwards_2).head(shape).copy()
jump_slow_right_backwards_2 = shuffle(jump_slow_right_backwards_2).head(shape).copy()
jump_slow_left_forward_2 = shuffle(jump_slow_left_forward_2).head(shape).copy()
jump_slow_right_forward_2 = shuffle(jump_slow_right_forward_2).head(shape).copy()

In [ ]:
jump_slow_2 = pd.concat([jump_slow_left_backwards_2, jump_slow_right_backwards_2, jump_slow_left_forward_2, jump_slow_right_forward_2])